# Team 3: Image Processing & Facial Recognition Model

This notebook demonstrates the complete pipeline for facial recognition, including:
1. Image collection and loading
2. Image augmentation and preprocessing
3. Feature extraction
4. Model training and evaluation
5. Prediction and authentication

## 1. Import Required Libraries

In [ ]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("All libraries imported successfully!")

## 2. Image Collection & Display

### 2.1 Load Member Images

First, ensure your images are organized in the following structure:
```
data/raw/images/
├── member1/
│   ├── neutral.jpg
│   ├── smiling.jpg
│   └── surprised.jpg
├── member2/
├── member3/
└── member4/
```

In [ ]:
def load_image(image_path):
    """Load an image from file."""
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    return image


def load_member_images(member_id, raw_images_path):
    """Load all images for a specific member."""
    member_path = os.path.join(raw_images_path, member_id)
    
    if not os.path.exists(member_path):
        raise FileNotFoundError(f"Member directory not found: {member_path}")
    
    images = {}
    for filename in os.listdir(member_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(member_path, filename)
            images[filename] = load_image(image_path)
    
    return images


def display_images(images_dict, title="Facial Images"):
    """Display multiple images in a grid."""
    num_images = len(images_dict)
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
    
    if num_images == 1:
        axes = [axes]
    
    for idx, (name, image) in enumerate(images_dict.items()):
        # Convert BGR to RGB for display
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        axes[idx].imshow(image_rgb)
        axes[idx].set_title(name)
        axes[idx].axis('off')
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


# Define paths
raw_images_path = "../data/raw/images"
processed_path = "../data/processed"
models_path = "../models"

print(f"Raw images path: {raw_images_path}")
print(f"Processed data path: {processed_path}")
print(f"Models path: {models_path}")

In [ ]:
# Try to load and display Alliance images
try:
    alliance_images = load_member_images("Alliance", raw_images_path)
    print(f"Successfully loaded {len(alliance_images)} images for Alliance")
    print(f"Images: {list(alliance_images.keys())}")
    display_images(alliance_images, "Alliance - Facial Images")
except FileNotFoundError as e:
    print(f"Note: {e}")
    print("Please ensure images are placed in data/raw/images/<member>/")

## 3. Image Augmentation

In [ ]:
def rotate_image(image, angle=15):
    """Rotate image by specified angle."""
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, matrix, (w, h))
    return rotated


def flip_image(image, axis=1):
    """Flip image horizontally or vertically."""
    return cv2.flip(image, axis)


def convert_to_grayscale(image):
    """Convert image to grayscale."""
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


def adjust_brightness(image, brightness_factor=0.7):
    """Adjust image brightness."""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[:, :, 2] = hsv[:, :, 2] * brightness_factor
    hsv[:, :, 2] = np.clip(hsv[:, :, 2], 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)


def apply_augmentations(image, augmentation_list):
    """Apply multiple augmentations to an image."""
    augmented_images = [image]  # Include original
    
    for aug_func in augmentation_list:
        augmented_images.append(aug_func(image))
    
    return augmented_images


print("Augmentation functions defined.")

In [ ]:
# Example: Apply augmentations to Alliance's first image
try:
    sample_image = list(alliance_images.values())[0]
    
    # Define augmentations
    augmentations = [
        ('Original', lambda img: img),
        ('Rotated +15°', lambda img: rotate_image(img, 15)),
        ('Rotated -15°', lambda img: rotate_image(img, -15)),
        ('Flipped', lambda img: flip_image(img)),
        ('Grayscale', lambda img: convert_to_grayscale(img)),
        ('Brightened', lambda img: adjust_brightness(img, 1.3))
    ]
    
    # Apply augmentations
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, (aug_name, aug_func) in enumerate(augmentations):
        aug_image = aug_func(sample_image)
        
        if len(aug_image.shape) == 3:
            aug_image_rgb = cv2.cvtColor(aug_image, cv2.COLOR_BGR2RGB)
            axes[idx].imshow(aug_image_rgb)
        else:
            axes[idx].imshow(aug_image, cmap='gray')
        
        axes[idx].set_title(aug_name)
        axes[idx].axis('off')
    
    plt.suptitle('Image Augmentations', fontsize=16)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Note: {e}")
    print("Augmentation functions are ready for use once images are loaded.")

## 4. Feature Extraction

In [ ]:
def extract_histogram_features(image, bins=32):
    """Extract color histogram features from image."""
    features = []
    for i in range(3):  # For each channel (B, G, R)
        hist = cv2.calcHist([image], [i], None, [bins], [0, 256])
        features.extend(hist.flatten())
    
    return np.array(features)


def extract_edge_features(image):
    """Extract edge features using Canny edge detection."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    return edges.flatten()


def extract_hog_features(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    """
    Extract HOG (Histogram of Oriented Gradients) features.
    More robust to lighting changes than basic edges.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog = cv2.HOGDescriptor(
        (64, 64),
        (16, 16),
        (8, 8),
        (8, 8),
        orientations
    )
    features = hog.compute(cv2.resize(gray, (64, 64)))
    return features.flatten()


def extract_color_moments(image):
    """Extract color moment features (mean, std, skewness per channel)."""
    features = []
    
    for i in range(3):  # For each channel
        channel = image[:, :, i].astype(np.float32)
        mean = np.mean(channel)
        std = np.std(channel)
        skewness = np.mean((channel - mean) ** 3) / (std ** 3 + 1e-6)
        
        features.extend([mean, std, skewness])
    
    return np.array(features)


def extract_lbp_features(image, num_points=8, radius=1):
    """
    Extract Local Binary Pattern (LBP) features.
    More invariant to lighting changes than histograms.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # LBP computation
    def lbp_computation(img, p=num_points, r=radius):
        rows, cols = img.shape
        lbp = np.zeros((rows, cols), dtype=np.uint8)
        
        for i in range(r, rows - r):
            for j in range(r, cols - r):
                center = img[i, j]
                lbp_val = 0
                
                for k in range(p):
                    angle = 2 * np.pi * k / p
                    x = int(r * np.cos(angle))
                    y = int(r * np.sin(angle))
                    neighbor = img[i + y, j + x]
                    lbp_val = (lbp_val << 1) | (1 if neighbor >= center else 0)
                
                lbp[i, j] = lbp_val
        
        return lbp
    
    lbp = lbp_computation(gray, num_points, radius)
    hist = cv2.calcHist([lbp], [0], None, [256], [0, 256])
    
    return hist.flatten()


def extract_all_features(image, feature_types=None):
    """
    Extract all specified features from an image.
    
    Feature types:
    - histogram: Color histogram (96 features)
    - edges: Canny edges (50176 features)
    - hog: Histogram of Oriented Gradients - MORE ROBUST (1764 features)
    - color_moments: Mean, std, skewness per channel (9 features)
    - lbp: Local Binary Patterns - MORE INVARIANT (256 features)
    """
    if feature_types is None:
        # Default: Use robust features
        feature_types = ['histogram', 'hog', 'color_moments', 'lbp']
    
    features = []
    
    if 'histogram' in feature_types:
        features.extend(extract_histogram_features(image))
    
    if 'edges' in feature_types:
        features.extend(extract_edge_features(image))
    
    if 'hog' in feature_types:
        features.extend(extract_hog_features(image))
    
    if 'color_moments' in feature_types:
        features.extend(extract_color_moments(image))
    
    if 'lbp' in feature_types:
        features.extend(extract_lbp_features(image))
    
    return np.array(features)


print("Advanced feature extraction functions defined.")

In [ ]:
# Example: Extract features from sample image
try:
    sample_image = list(alliance_images.values())[0]
    
    # Extract features using advanced methods
    features = extract_all_features(sample_image)
    
    print(f"Sample Image Shape: {sample_image.shape}")
    print(f"Total Features Extracted: {len(features)}")
    print(f"Feature Vector Shape: {features.shape}")
    print(f"\nFeature Statistics:")
    print(f"  Min: {features.min():.4f}")
    print(f"  Max: {features.max():.4f}")
    print(f"  Mean: {features.mean():.4f}")
    print(f"  Std: {features.std():.4f}")
    print(f"\nFeatures included:")
    print(f"  ✓ Histogram (Color distribution)")
    print(f"  ✓ HOG (Histogram of Oriented Gradients - lighting invariant)")
    print(f"  ✓ Color Moments (Mean, std, skewness per channel)")
    print(f"  ✓ LBP (Local Binary Patterns - invariant to monotonic brightness)")
    
except Exception as e:
    print(f"Note: {e}")
    print("Feature extraction functions are ready for use once images are loaded.")

## 5. Data Preparation & Model Training

In [ ]:
def prepare_training_data(raw_images_path, members=['Alliance', 'Ange', 'Elissa', 'Terry']):
    """Prepare training data from raw images using advanced features."""
    X_list = []
    y_list = []
    label_encoder = {}
    reverse_label_encoder = {}
    
    # Create label encoder
    for idx, member in enumerate(members):
        label_encoder[member] = idx
        reverse_label_encoder[idx] = member
    
    print("Loading and processing images...")
    print("Feature types: Histogram + HOG + Color Moments + LBP")
    
    for member_id in members:
        print(f"Processing {member_id}...")
        
        try:
            # Load member's images
            member_images = load_member_images(member_id, raw_images_path)
            
            for img_name, image in member_images.items():
                # Resize image to standard size for consistency
                image_resized = cv2.resize(image, (224, 224))
                
                # Extract advanced features from original image
                feature_vector = extract_all_features(image_resized)
                if feature_vector is not None and len(feature_vector) > 0:
                    X_list.append(feature_vector)
                    y_list.append(label_encoder[member_id])
                
                # Apply augmentations and extract features
                augmentations = [
                    lambda img: rotate_image(img, 15),
                    lambda img: flip_image(img),
                    lambda img: convert_to_grayscale(img)
                ]
                
                augmented_images = apply_augmentations(image_resized, augmentations)
                
                for aug_img in augmented_images[1:]:  # Skip original
                    # Ensure image is RGB
                    if len(aug_img.shape) == 2:  # Grayscale
                        aug_img = cv2.cvtColor(aug_img, cv2.COLOR_GRAY2BGR)
                    elif aug_img.shape[2] == 4:  # RGBA
                        aug_img = cv2.cvtColor(aug_img, cv2.COLOR_RGBA2BGR)
                    
                    # Ensure consistent size
                    aug_img = cv2.resize(aug_img, (224, 224))
                    
                    feature_vector = extract_all_features(aug_img)
                    if feature_vector is not None and len(feature_vector) > 0:
                        X_list.append(feature_vector)
                        y_list.append(label_encoder[member_id])
        
        except FileNotFoundError as e:
            print(f"Warning: {e}")
            continue
    
    # Convert to numpy array with consistent shape
    if len(X_list) > 0:
        # Ensure all feature vectors have the same length
        feature_length = len(X_list[0])
        X_list = [feat for feat in X_list if len(feat) == feature_length]
        X = np.array(X_list)
    else:
        X = np.array([])
    
    y = np.array(y_list)
    
    print(f"\nTraining data shape: {X.shape}")
    print(f"Number of samples: {len(y)}")
    if len(X_list) > 0:
        print(f"Feature vector length: {len(X_list[0])}")
        print(f"Features per image: {len(X_list[0])} (Advanced: HOG + LBP + Histogram + Color Moments)")
    
    return X, y, label_encoder, reverse_label_encoder


print("Advanced data preparation function defined.")

In [ ]:
# Prepare training data
try:
    X, y, label_encoder, reverse_label_encoder = prepare_training_data(raw_images_path)
    
    print("\n" + "="*50)
    print("TRAINING DATA SUMMARY")
    print("="*50)
    print(f"Total samples: {len(X)}")
    print(f"Feature dimensions: {X.shape[1]}")
    print(f"Number of classes: {len(label_encoder)}")
    print("\nClass distribution:")
    unique, counts = np.unique(y, return_counts=True)
    for label, count in zip(unique, counts):
        member = reverse_label_encoder[label]
        print(f"  {member}: {count} samples")
    print("="*50)
    
except Exception as e:
    print(f"Note: {e}")
    print("Data preparation functions are ready for use once images are properly organized.")

In [ ]:
# Train the model with advanced features
try:
    print("Training Facial Recognition Model (with Advanced Features)...")
    print("-" * 60)
    print("Feature extraction: HOG + LBP + Histogram + Color Moments")
    print("-" * 60)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\nTraining set size: {len(X_train)} samples")
    print(f"Test set size: {len(X_test)} samples")
    print(f"Feature dimension: {X.shape[1]}")
    
    # Train Random Forest model
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=20,
        random_state=42,
        n_jobs=-1
    )
    
    print("\nTraining Random Forest Classifier...")
    model.fit(X_train, y_train)
    print("✓ Model training completed!")
    
    # Evaluate
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_train = f1_score(y_train, y_pred_train, average='weighted')
    f1_test = f1_score(y_test, y_pred_test, average='weighted')
    
    print("\n" + "="*60)
    print("MODEL PERFORMANCE (With Advanced Features)")
    print("="*60)
    print(f"Training Accuracy: {accuracy_train:.4f} ({accuracy_train*100:.2f}%)")
    print(f"Testing Accuracy:  {accuracy_test:.4f} ({accuracy_test*100:.2f}%)")
    print(f"Training F1-Score: {f1_train:.4f}")
    print(f"Testing F1-Score:  {f1_test:.4f}")
    print("="*60)
    print("\nNote: These metrics use robust features (HOG + LBP)")
    print("Expected to handle lighting/angle/environment variations better")
    print("="*60)
    
except Exception as e:
    print(f"Note: {e}")
    print("Model training will proceed once training data is prepared.")

## 6. Model Evaluation

In [ ]:
try:
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred_test)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=list(reverse_label_encoder.values()),
                yticklabels=list(reverse_label_encoder.values()))
    plt.title('Confusion Matrix - Facial Recognition Model')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()
    
    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_test,
                              target_names=list(reverse_label_encoder.values())))
    
except Exception as e:
    print(f"Note: {e}")
    print("Evaluation will be performed after model training.")

## 7. Save Features and Model

In [ ]:
import pickle
import os

try:
    # Create directories if they don't exist
    os.makedirs(processed_path, exist_ok=True)
    os.makedirs(models_path, exist_ok=True)
    
    # Save model
    model_path = os.path.join(models_path, 'facial_recognition_model.pkl')
    with open(model_path, 'wb') as f:
        pickle.dump({
            'model': model,
            'label_encoder': label_encoder,
            'reverse_label_encoder': reverse_label_encoder
        }, f)
    print(f"Model saved to: {model_path}")
    
    # Save features to CSV
    features_df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
    features_df['label'] = y
    features_df['member'] = features_df['label'].map(reverse_label_encoder)
    
    csv_path = os.path.join(processed_path, 'image_features.csv')
    features_df.to_csv(csv_path, index=False)
    print(f"Features saved to: {csv_path}")
    print(f"Shape: {features_df.shape}")
    
except Exception as e:
    print(f"Note: {e}")
    print("Files will be saved once model training is complete.")

## 8. Test Predictions

In [ ]:
def predict_face(image_path, model, reverse_label_encoder, target_size=(224, 224)):
    """Predict the user from a facial image."""
    try:
        # Load image
        image = load_image(image_path)
        
        # Resize to standard size
        image_resized = cv2.resize(image, target_size)
        
        # Extract features
        features = extract_all_features(image_resized).reshape(1, -1)
        
        # Make prediction
        prediction = model.predict(features)[0]
        probabilities = model.predict_proba(features)[0]
        confidence = np.max(probabilities)
        
        member = reverse_label_encoder[prediction]
        
        return {
            'member': member,
            'confidence': confidence,
            'probabilities': probabilities
        }
    except Exception as e:
        print(f"Error: {e}")
        return None


print("Prediction function defined.")

In [ ]:
# Test prediction on a sample image
try:
    test_image_path = os.path.join(raw_images_path, 'Alliance', list(alliance_images.keys())[0])
    
    result = predict_face(test_image_path, model, reverse_label_encoder)
    
    if result:
        print("\nPrediction Result:")
        print(f"  Predicted Member: {result['member']}")
        print(f"  Confidence: {result['confidence']:.4f}")
        print(f"\n  Probabilities for each member:")
        for member, prob in zip(reverse_label_encoder.values(), result['probabilities']):
            print(f"    {member}: {prob:.4f}")
    
except Exception as e:
    print(f"Note: {e}")
    print("Predictions will be available after model training.")

In [ ]:
## 9. Authentication & Unauthorized Access Detection

### 9.1 Define Confidence Threshold and Authentication Logic

# Set confidence threshold for authorization
CONFIDENCE_THRESHOLD = 0.70  # Minimum 70% confidence to grant access

def authenticate_user(image_path, model, reverse_label_encoder, confidence_threshold=CONFIDENCE_THRESHOLD):
    """
    Authenticate a user based on facial recognition with confidence threshold.
    
    Args:
        image_path: Path to the facial image
        model: Trained facial recognition model
        reverse_label_encoder: Mapping from label to member name
        confidence_threshold: Minimum confidence required for authorization
    
    Returns:
        Dictionary with authentication result
    """
    result = predict_face(image_path, model, reverse_label_encoder)
    
    if result is None:
        return {
            'authorized': False,
            'reason': 'Error: Could not process image',
            'member': 'Unknown',
            'confidence': 0.0
        }
    
    # Check if confidence meets threshold
    if result['confidence'] >= confidence_threshold:
        return {
            'authorized': True,
            'reason': f" AUTHORIZED - {result['member']} recognized with {result['confidence']*100:.2f}% confidence",
            'member': result['member'],
            'confidence': result['confidence']
        }
    else:
        return {
            'authorized': False,
            'reason': f" DENIED - Confidence too low ({result['confidence']*100:.2f}% < {confidence_threshold*100:.0f}% threshold)",
            'member': result['member'],
            'confidence': result['confidence']
        }


print("Authentication logic defined.")
print(f"Confidence Threshold: {CONFIDENCE_THRESHOLD*100:.0f}%")

In [ ]:
### 9.2 Comprehensive Authentication Tests

print("\n" + "="*70)
print("FACIAL RECOGNITION AUTHENTICATION SYSTEM - COMPREHENSIVE TESTS")
print("="*70)

# TEST 1: Authorized User
print("\n" + "="*70)
print("TEST 1: AUTHORIZED USER - Known Member (Training Distribution)")
print("="*70)

try:
    # Use an authorized member's image from training distribution
    authorized_image_path = os.path.join(raw_images_path, 'Alliance', list(alliance_images.keys())[0])
    
    auth_result = authenticate_user(authorized_image_path, model, reverse_label_encoder)
    
    print(f"\nImage Path: {authorized_image_path}")
    print(f"Predicted Member: {auth_result['member']}")
    print(f"Confidence: {auth_result['confidence']:.4f} ({auth_result['confidence']*100:.2f}%)")
    print(f"Threshold: {CONFIDENCE_THRESHOLD*100:.0f}%")
    print(f"\nDecision: {auth_result['reason']}")
    
    if auth_result['authorized']:
        print(" ACCESS GRANTED - User can proceed to voice verification")
    else:
        print(" ACCESS DENIED - User cannot proceed")
    
except Exception as e:
    print(f"Note: {e}")
    print("Test will run once authorized image is available.")

In [ ]:
### TEST2: Test Unauthorized/Unknown Access

print("\n" + "="*60)
print("TEST 2: UNAUTHORIZED USER - Unknown/Suspicious Face")
print("="*60)
print("\nNote: To test unauthorized access, use:")
print("  - An unknown person's face (not in training data)")
print("  - A heavily disguised version of a known member")
print("  - A low-quality or partially obscured face")
print("\nExample (if you have unauthorized image):")
print("  unauthorized_image_path = '../data/raw/images/unauthorized/unknown.jpg'")
print("  result = authenticate_user(unauthorized_image_path, model, reverse_label_encoder)")

try:
    # load an unauthorized image if it exists
    
    unauthorized_paths = [
        os.path.join(raw_images_path, 'unauthorized', 'unknown6.jpg'),
        os.path.join(raw_images_path, 'unknown1.jpg'),
    ]
    
    unauthorized_image_path = None
    for path in unauthorized_paths:
        if os.path.exists(path):
            unauthorized_image_path = path
            break
    
    if unauthorized_image_path:
        print(f"\nFound unauthorized image at: {unauthorized_image_path}")
        
        unauth_result = authenticate_user(unauthorized_image_path, model, reverse_label_encoder)
        
        print(f"\nImage Path: {unauthorized_image_path}")
        print(f"Closest Match: {unauth_result['member']}")
        print(f"Confidence: {unauth_result['confidence']:.4f} ({unauth_result['confidence']*100:.2f}%)")
        print(f"Threshold: {CONFIDENCE_THRESHOLD*100:.0f}%")
        print(f"\nDecision: {unauth_result['reason']}")
        
        if unauth_result['authorized']:
            print("\n WARNING - Unknown user was authorized (confidence too high)")
        else:
            print("\n SECURITY OK - Unknown user was correctly rejected")
            print("   User cannot proceed with transaction")
    else:
        print("\n" + "-"*60)
        print("No unauthorized image found.")
        print("To test, create a folder and add an unknown person's image:")
        print("  1. Create: data/raw/images/unauthorized/")
        print("  2. Add image: unknown.jpg or similar")
        print("  3. Re-run this cell")
        print("-"*60)

except Exception as e:
    print(f"Note: {e}")
    print("Unauthorized test will run when test image is available.")

1. USER SUBMITS FACIAL IMAGE
   ↓
2. FACIAL RECOGNITION MODEL PROCESSES IMAGE
   ↓
3. MODEL EXTRACTS FEATURES & MAKES PREDICTION
   ↓
4. MODEL RETURNS PREDICTION + CONFIDENCE SCORE
   ↓
5. CHECK CONFIDENCE AGAINST THRESHOLD (70%)
   │
   ├─ If Confidence >= 70% → ✅ AUTHORIZED
   │  └─ Proceed to Voice Verification Stage
   │
   └─ If Confidence < 70% → ❌ DENIED
      └─ Access Denied - End Transaction
   ↓
6. DISPLAY RESULT TO USER

## Summary

This notebook demonstrates the complete facial recognition pipeline:

### Key Steps:
1. **Image Collection**: Loaded facial images from each team member
2. **Augmentation**: Applied various augmentations to increase training data
3. **Feature Extraction**: Extracted histogram, edge, and color moment features
4. **Model Training**: Trained a Random Forest classifier
5. **Evaluation**: Evaluated using Accuracy, F1-Score, and Confusion Matrix
6. **Prediction**: Made predictions on test images

### Output Files:
- `image_features.csv`: All extracted features
- `facial_recognition_model.pkl`: Trained model

### Performance Metrics:
- Accuracy
- F1-Score
- Confusion Matrix
- Classification Report